In [30]:
import numpy as np
import pandas as pd
import sys
from sklearn import ensemble as em
train = pd.read_csv('year1-data.csv')
train2  = pd.read_csv('year2-data.csv')
X_test = pd.read_csv('testing_data.csv')
#train.shape #6588(366*18)*26
#train2.shape #6570(365*18)*26
#X_test.shape #9000*11

In [31]:
#train = train.replace("#","")
#train = train.replace("\*","")
#train = train.replace("x","")
def preprocess(train):
    train.replace("#",'',inplace=True,regex=True)
    train.replace("x",'',inplace=True,regex=True)
    train.replace("\*",'',inplace=True,regex=True)
    train.replace("NR",0,inplace=True) #將NR變成0
    train.fillna(0, inplace=True) #把NAN變成0
    train = train.iloc[:,2:]
    return train

In [32]:
train =  preprocess(train) #6588*24
train2 = preprocess(train2) #6570*24
X_test = preprocess(X_test) #9000*9


In [33]:
def get_feature(train,test=False):
    train = np.array(train,dtype=float)# to array 6588*24
    tmp=[]
    feature = []
    for i  in range(train.shape[0]): #0~6587
        if not test:
            if i/18>180 and i/18<240:
                continue
        index = i%18
        if index==2 or index==4 or index==5 or index==7 or index==8 or index==9 or index==12:#只取CO,NO,NO2,PM2.5,PM10,O3,SO2
                        
    
            tmp.append(train[i])
        if index==17:
            tmp = np.array(tmp,dtype=float)#shape7*24
            feature.append(tmp)
            tmp = []
    feature = np.array(feature,dtype=float)#366*168
    print(feature.shape)
    return feature

In [34]:
train = get_feature(train)#shape 366*168
train2 = get_feature(train2)#shape 365*168
X_test = get_feature(X_test,test=True)#500*7*9


(306, 7, 24)
(305, 7, 24)
(500, 7, 9)


In [35]:
train = np.concatenate((train, train2), axis=0)#shape 731*7*24

In [36]:
tmp = train[0]
tmp.shape
for i in range(1,train.shape[0]):
    tmp =  np.concatenate((tmp, train[i]), axis=1)
train = tmp #7*17544

In [37]:
def valid(x, y):
    if y <= 2 or y > 100:
        return False
    for i in range(9):
        if x[5,i] <= 2 or x[5,i] > 100:
            return False
    for i in range(9):
        for j in range(7):
            if x[j,i] < 0:
                return False
    return True

In [38]:
tmp = []
X_test.shape
for i in range(X_test.shape[0]):
    tmp.append(X_test[i].flatten())
X_test = tmp 
X_test = np.array(X_test,dtype=float)#shape 500*63
X_test.shape


(500, 63)

In [39]:
X=[]
Y=[]
for i in range(0, train.shape[1]-9):
    x =  train[:, i:i+9]
    y = train[5, i+9]
    if valid(x,y):
        X.append(x.flatten())
        Y.append([y])

X = np.array(X)#shape 5845*63
Y = np.array(Y)#shape 5845*1
#X_1=X[0:2327,:]
#X_2=X[2327:4654,:]
#X_3=X[4654:6981,:]
#X_4=X[6981:9308,:]
#X_5=X[9308:,:]
#Y_1 = Y[0:2327,:]
#Y_2 = Y[2327:4654,:]
#Y_3 = Y[4654:6981,:]
#Y_4 = Y[6981:9308,:]
#Y_5 = Y[9308:,:]


In [40]:
class Linear_Regression():
    def __init__(self,B = np.double(0),W = np.ones((63, 1))):
        self.B = B
        self.W = W
        pass
    def error(self, X, Y):
        return Y - self.predict(X)

    def MSE(self, X, Y):#MSE(X) = E[(X-Y)^2]
        return np.sqrt(np.mean(self.error(X, Y) ** 2))
    
    def predict(self, X):#   y = wx+b
        _X = np.reshape(X, (-1, self.feature_dim))
        return np.dot(_X, self.W) + self.B

    def fit(self, _X, Y, valid, max_epoch=500000, lr=0.000001, C=0.0):
        assert _X.shape[0] == Y.shape[0]
        N = _X.shape[0]
        self.feature_dim = feature_dim = _X.shape[1]
        self.C = C
        print(_X.shape)
        X = _X
        
     
        #self.B = 0.0
        #self.W = np.ones((self.feature_dim, 1))
        for epoch in range(1, max_epoch+1):
            
            error = Y - self.predict(X)          
            B_grad = -np.sum(error)  / N
            W_grad = -np.dot(X.T, error) / N
            self.B = self.B - lr  * B_grad
            self.W = self.W - lr  * W_grad
            
            if epoch % 1000 == 0:
                print('[Epoch {}]: loss: {}'.format(epoch, self.MSE(X, Y)))

In [41]:
model = Linear_Regression()
model.fit(X, Y, valid=None, max_epoch=100000, lr=0.000003, C=0.00001)

(11635, 63)
[Epoch 1000]: loss: 21.205522703381693
[Epoch 2000]: loss: 15.870487303452933
[Epoch 3000]: loss: 12.803205960906885
[Epoch 4000]: loss: 10.712575638390904
[Epoch 5000]: loss: 9.175346610722588
[Epoch 6000]: loss: 8.00854585618434
[Epoch 7000]: loss: 7.115023013367047
[Epoch 8000]: loss: 6.432698911924571
[Epoch 9000]: loss: 5.915942219128102
[Epoch 10000]: loss: 5.528468077275539
[Epoch 11000]: loss: 5.240636435721716


KeyboardInterrupt: 

In [12]:
predict = model.predict(X_test)
for i in range(len(predict)):
    if predict[i]<0:
        predict[i] = 19.445

In [13]:
 with open('line_regression_result.csv', 'w') as f:
        print('id,value', file=f)
        for (i, p) in enumerate(predict) :
            print('id_{},{}'.format(i, p), file=f)

# TODO #

* ~~去掉七八月~~
* ~~shift 3 小時~~
* ~~只做PM2.5 PM10 NO2影響大......~~
* ~~負的資料砍掉~~
* n-fold
* normalize
* gradient bootsing

